In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
dataset, info = tfds.load(
    "cats_vs_dogs",
    with_info=True,
    as_supervised=True
)

total_samples = info.splits['train'].num_examples
print("Total samples:", total_samples)


In [ ]:
train_size = int(0.7 * total_samples)
val_size = int(0.15 * total_samples)

train_ds = dataset['train'].take(train_size)
temp_ds = dataset['train'].skip(train_size)

val_ds = temp_ds.take(val_size)
test_ds = temp_ds.skip(val_size)


In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 32

def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = image / 255.0
    return image, label

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.2)
])

train_ds = train_ds.map(preprocess).map(
    lambda x, y: (data_augmentation(x), y)
)

val_ds = val_ds.map(preprocess)
test_ds = test_ds.map(preprocess)

train_ds = train_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [ ]:
baseline_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(224,224,3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

baseline_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

baseline_model.fit(train_ds, validation_data=val_ds, epochs=5)

baseline_acc = baseline_model.evaluate(test_ds)[1]
print("Baseline Test Accuracy:", baseline_acc)


In [ ]:
base_model = tf.keras.applications.ResNet50(
    weights="imagenet",
    include_top=False,
    input_shape=(224,224,3)
)

base_model.trainable = False


In [ ]:
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(256, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(1, activation="sigmoid")(x)

model = tf.keras.Model(inputs=base_model.input, outputs=output)


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.fit(train_ds, validation_data=val_ds, epochs=5)


In [ ]:
for layer in base_model.layers[-30:]:
    layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.fit(train_ds, validation_data=val_ds, epochs=5)


In [ ]:
test_loss, test_acc = model.evaluate(test_ds)
print("Transfer Learning Test Accuracy:", test_acc)


In [ ]:
y_true = []
y_pred = []

for images, labels in test_ds:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend((preds > 0.5).astype(int))

print(classification_report(y_true, y_pred))
print(confusion_matrix(y_true, y_pred))


In [ ]:
last_conv_layer = "conv5_block3_out"

grad_model = tf.keras.models.Model(
    model.inputs,
    [model.get_layer(last_conv_layer).output, model.output]
)

image, label = next(iter(test_ds))
img = image[0]

with tf.GradientTape() as tape:
    conv_outputs, predictions = grad_model(tf.expand_dims(img, axis=0))
    loss = predictions[:, 0]

grads = tape.gradient(loss, conv_outputs)
pooled_grads = tf.reduce_mean(grads, axis=(0,1,2))

conv_outputs = conv_outputs[0]
heatmap = tf.reduce_sum(conv_outputs * pooled_grads, axis=-1)
heatmap = tf.maximum(heatmap, 0) / tf.reduce_max(heatmap)


In [ ]:
heatmap = cv2.resize(heatmap.numpy(), (224,224))
heatmap = np.uint8(255 * heatmap)
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

superimposed_img = heatmap * 0.4 + img.numpy() * 255

plt.imshow(superimposed_img.astype("uint8"))
plt.title("Grad-CAM Visualization")
plt.axis("off")
plt.show()


In [ ]:
model.save("cats_vs_dogs_resnet50.h5")


In [ ]:
from google.colab import files
files.download("cats_vs_dogs_resnet50.h5")
